參考:https://github.com/aws-samples/amazon-forecast-samples/blob/master/notebooks/basic/Tutorial/1.Importing_Your_Data.ipynb

In [1]:
import sys
import os
import pandas as pd

%reload_ext autoreload
import boto3
import s3fs

In [ ]:
region = 'ap-northeast-1'
BUCKET_NAME = ''

In [ ]:
# Create API Session
session = boto3.Session(region_name=region)
forecast = session.client(service_name='forecast')
forecastquery = session.client(service_name='forecastquery')